In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df_pred_auction = pd.DataFrame()
df_pred_transaction = pd.DataFrame()
recommend_df = pd.DataFrame()

In [ ]:
df = pd.read_csv('sample.csv')
#len(df.dropna(axis=1))
df.isnull().sum()

In [ ]:
xy = np.loadtxt('ML_data2.csv',delimiter=',',dtype=np.float32)

scaler = MinMaxScaler()
xy2 = xy

x_data = xy2[:,1:-1]
y_data = xy2[:,[-1]]/1000000000

scaler.fit(x_data)
x_data = scaler.transform(x_data)

x_train, x_test, y_train, y_test = train_test_split(x_data,y_data,random_state=123)

x = tf.placeholder(tf.float32,shape=[None,10],name='x')
y = tf.placeholder(tf.float32,shape=[None,1],name='y')

w = tf.Variable(tf.random_normal([10,1]),dtype=tf.float32,name='w')
b = tf.Variable(tf.random_normal([1]),dtype=tf.float32,name='b')

hypothesis = tf.matmul(x,w) + b

cost = tf.reduce_mean(tf.square(hypothesis - y),name='cost')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0004,name='optimizer')
train = optimizer.minimize(cost)

#saver = tf.compat.v1.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())


#coord = tf.train.Coordinator()
#threads = tf.train.start_queue_runners(sess=sess, coord=coord)

feed_train = {x:x_train,y:y_train}
feed_test = {x:x_test,y:y_test}
for step in range(500001):
    c2,h2,_ = sess.run([cost,hypothesis,train],
                    feed_dict=feed_train)
    
    if step%1000 == 0:
        print(step,'cost:',c2,'\npredict:',h2)
        
        
        
#saver = tf.train.Saver()
#save_path = saver.save(sess, 'model/auction/lys_auction_model_1_0.ckpt')
#joblib.dump(scaler, 'model/auction/a_scaler.gz')

In [ ]:
def auction(df):
    import joblib
    import tensorflow as tf
    
    gu_cd = pd.read_csv("model/auction/a_gu_cd.csv")
    dong_cd = pd.read_csv("model/auction/a_dong_cd.csv")
    
    # 구, 동 이름 숫자 코딩
    for i in df.index:
        df.loc[i,"gu_num"] = int(gu_cd.loc[gu_cd.gu==df.loc[i,"gu"],"gu_num"])
        df.loc[i,"dong_num"] = int(dong_cd.loc[dong_cd.dong==df.loc[i,"dong"],"dong_num"])

    # 필요한 열 뽑기
    x_data = df[['lowest','price','low_per','gu_num','dong_num','date_ym','floor','build_area','comment_val','case_year']].dropna()
    #print(x_data)
    # 스케일링
    scaler = joblib.load("model/auction/a_scaler.gz")
    x_data = scaler.transform(x_data)
    #print(x_data)
    # 모델 불러오기  
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.import_meta_graph("model/auction/lys_auction_model_1_0.ckpt.meta")
    saver.restore(sess,"model/auction/lys_auction_model_1_0.ckpt")  
    graph = tf.get_default_graph()
    
    x = graph.get_tensor_by_name("x:0")
    
    w = graph.get_tensor_by_name("w:0")
    b = graph.get_tensor_by_name("b:0")
    
    hypot = tf.matmul(x,w)+b
    cost = graph.get_tensor_by_name("cost:0")
    print(cost)
#     optimizer = graph.get_operation_by_name("optimizer:0")
    
    print(hypot)
    
    # 낙찰가 예측
    h = sess.run(hypot, feed_dict={x:x_data})

    return h.flatten() 

In [ ]:
y = auction(df)*1000000000
y
#y = transaction(df)
y.haed(30)

In [ ]:
y = y[:30]

In [ ]:
cnt = 0
for i,j in zip(df['hammer'],y):
    cnt += 1
    print(cnt,'낙찰가:',i,'','예상가:',int(j))

In [ ]:
plt.scatter(df['hammer'],y)

In [ ]:
def transaction(df):
    from tensorflow.keras.models import load_model
    from sklearn.preprocessing import MinMaxScaler
    import joblib
    
    gu_cd = pd.read_csv("model/transaction/t_gu_cd.csv")
    dong_cd = pd.read_csv("model/transaction/t_dong_cd.csv")
#    brand_cd = pd.read_csv("model/transaction/t_brand_cd.csv")
    
    # 구, 동, 아파트 이름 숫자 코딩, 사용연수 추가 -> 아파트이름, 사용연수 제외
    for i in df.index:
        try:
            df.loc[i,"구_int"] = int(gu_cd.loc[gu_cd.구==df.loc[i,"gu"],"구_int"])
            df.loc[i,"동_int"] = int(dong_cd.loc[dong_cd.동==df.loc[i,"dong"],"동_int"])
#            df.loc[i,"brand_int"] = int(brand_cd.loc[(brand_cd.구==df.loc[i,"gu"])&(brand_cd.동==df.loc[i,"dong"])&(brand_cd.brand_name==df.loc[i,"brand_name"]),"brand_name_int"])
#            df.loc[i,"years_use"] = int(brand_cd.loc[(brand_cd.구==df.loc[i,"gu"])&(brand_cd.동==df.loc[i,"dong"])&(brand_cd.brand_name==df.loc[i,"brand_name"]),"years_use"])
        
        except TypeError:
            pass
    
    # 계약년월 받아서 추가   
    while True:
        date_ym = input("예상 매매날짜(연,월)를 YYYYMM 형태로 기입하세요.")
        
        if len(date_ym) > 6:
            print("날짜를 연월(YYYYMM) 형태로 기입해주세요.(일 제외) 예) 202210")
            continue
            
        elif (int(date_ym[:4]) < 2100) & (int(date_ym[4:]) <= 12):
            date_ym = int(date_ym)
            break

        else:
            print("날짜를 확인해주세요.")
            continue
        
    df["계약년월"] = date_ym

    # 필요한 열 뽑기
    x_data = df[["계약년월","구_int","동_int","build_area","floor"]]
    
    print(x_data)
#     x_data = x_data.dropna()
    
    # 모델 불러오기
    model = load_model("model/transaction/model5.h5")
#    print(mse, mae)
    
    # 스케일링
    scaler = joblib.load("model/t_scaler.gz")
    x_data = scaler.transform(x_data)
    
    # 실거래가 예측
    y = model.predict(x_data)

    return y.flatten()

In [ ]:
df = pd.read_csv('../data/auction/auction_pjs3.csv')
a_a = transaction(df)
#x = x[numpy.logical_not(numpy.isnan(x))]

In [ ]:
output1 = a_a[np.logical_not(np.isnan(a_a))]
#plt.scatter(df['price'],a_a)
print(len(output1))

In [ ]:
auc_pred = df_pred_auction['y1']
auc_cost = df_pred_auction['cost1']*1000000000
trans_pred = df_pred_transaction['y2']
trans_cost = df_pred_transaction['cost2']*1000000000

In [141]:
# 매물추천 프로그램 함수화
# 입력 값에 따라서 데이터 프레임 개인의 고유 데이터 프레임 생성


def type_items(type_):
    
    box = []
    
    for step in range(len(recommend_df)):
        if recommend_df['recommend'][step] in type_:
            box.append(recommend_df['y1'][step])
            print(recommend_df[['addr_name','brand_name']][step],
                    '예상낙찰가:{} ,예상차익:{}, 예산안:{}, 예상경쟁률:{}'.format(recommend_df['y1'][step],recommend_df['diff_high'][step],recommend_df['pred_budget'][step],recommend_df['commpetition']))
            print(recommend[['hammer','y1','y2']])
            if len(box) == 0:
                print('확인된 매물이 없습니다.')
            
def soul_pull(soul):
    if soul == '2':
        
            return False
            
    elif soul == '1':
        for step in range(len(recommend_df)):
            if recommend_df['soul'][step] in '영끌추천':
                print('예상 수이익:',recommend_df['diff_high'][step],'영끌추천')
                print(recommend[['hammer','y1','y2']])    
        print('최저금리로 5분대출 전화한통일면 OK \n070-0000-0000 언제나 열려있습니다. \n가족처럼 모시겠습니다.\n\n\n')
        end_ = input('종료하시겠습니까?')
        print('종료하시려면 2번을 누르세요. \n처음으로 돌아가시려면 아무키나 누르세요.')
        
        if end_ in '2':
            
            return False

        
        
def auction(df):
    import joblib
    import tensorflow as tf
    
    gu_cd = pd.read_csv("model/auction/a_gu_cd.csv")
    dong_cd = pd.read_csv("model/auction/a_dong_cd.csv")
    
    # 구, 동 이름 숫자 코딩
    for i in df.index:
        df.loc[i,"gu_num"] = int(gu_cd.loc[gu_cd.gu==df.loc[i,"gu"],"gu_num"])
        df.loc[i,"dong_num"] = int(dong_cd.loc[dong_cd.dong==df.loc[i,"dong"],"dong_num"])

    # 필요한 열 뽑기
    x_data = df[['lowest','price','low_per','gu_num','dong_num','date_ym','floor','build_area','comment_val','case_year']].dropna()
    
    # 스케일링
    scaler = joblib.load("model/auction/a_scaler.gz")
    x_data = scaler.transform(x_data)

    # 모델 불러오기  
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.import_meta_graph("model/auction/lys_auction_model_1_0.ckpt.meta")
    saver.restore(sess,"model/auction/lys_auction_model_1_0.ckpt")  
    graph = tf.get_default_graph()
    
    x = graph.get_tensor_by_name("x:0")
    
    w = graph.get_tensor_by_name("w:0")
    b = graph.get_tensor_by_name("b:0")
    
    hypot = tf.matmul(x,w)+b
    cost = graph.get_tensor_by_name("cost:0")
#     optimizer = graph.get_operation_by_name("optimizer:0")
    
    print(hypot)
    
    # 낙찰가 예측
    h = sess.run(hypot, feed_dict={x:x_data})

    return h.flatten() 



def transaction(df):
    from tensorflow.keras.models import load_model
    from sklearn.preprocessing import MinMaxScaler
    import joblib
    
    gu_cd = pd.read_csv("model/transaction/t_gu_cd.csv")
    dong_cd = pd.read_csv("model/transaction/t_dong_cd.csv")
#    brand_cd = pd.read_csv("model/transaction/t_brand_cd.csv")
    
    # 구, 동, 아파트 이름 숫자 코딩, 사용연수 추가 -> 아파트이름, 사용연수 제외
    for i in df.index:
        try:
            df.loc[i,"구_int"] = int(gu_cd.loc[gu_cd.구==df.loc[i,"gu"],"구_int"])
            df.loc[i,"동_int"] = int(dong_cd.loc[dong_cd.동==df.loc[i,"dong"],"동_int"])
#            df.loc[i,"brand_int"] = int(brand_cd.loc[(brand_cd.구==df.loc[i,"gu"])&(brand_cd.동==df.loc[i,"dong"])&(brand_cd.brand_name==df.loc[i,"brand_name"]),"brand_name_int"])
#            df.loc[i,"years_use"] = int(brand_cd.loc[(brand_cd.구==df.loc[i,"gu"])&(brand_cd.동==df.loc[i,"dong"])&(brand_cd.brand_name==df.loc[i,"brand_name"]),"years_use"])
        
        except TypeError:
            pass
    
    # 계약년월 받아서 추가   
    while True:
        date_ym = input("예상 매매날짜(연,월)를 YYYYMM 형태로 기입하세요.")
        
        if len(date_ym) > 6:
            print("날짜를 연월(YYYYMM) 형태로 기입해주세요.(일 제외) 예) 202210")
            continue
            
        elif (int(date_ym[:4]) < 2100) & (int(date_ym[4:]) <= 12):
            date_ym = int(date_ym)
            break

        else:
            print("날짜를 확인해주세요.")
            continue
        
    df["계약년월"] = date_ym

    # 필요한 열 뽑기
    x_data = df[["계약년월","구_int","동_int","build_area","floor"]]
    
    #print(x_data)
    #x_data = x_data.dropna()
    
    # 모델 불러오기
    model = load_model("model/transaction/model5.h5")
#    print(mse, mae)
    
    # 스케일링
    scaler = joblib.load("model/transaction/t_scaler.gz")
    x_data = scaler.transform(x_data)
    
    # 실거래가 예측
    y = model.predict(x_data)

    return y.flatten()



def recommend_apt(data):
    import pandas as pd
    import numpy as np
    
    global recommend_df
    
    recommend_df = pd.DataFrame(data)
    
    recommend_df['y1'] = auction(recommend_df)                                # 경매가 예측 함수 호출
    recommend_df['cost1'] = 0.011388925                                       # 예측가격 컬럼 생성
    
    auc_pred = recommend_df['y1']
    auc_cost = recommend_df['cost1']
    
    recommend_df['y2'] = transaction(df)
    recommend_df['cost2'] = 0.02639
    
    trans_pred = recommend_df['y2']
    trans_cost = recommend_df['cost2']
    
    formula1_p = (auc_pred + auc_cost)
    formula1_m = (auc_pred - auc_cost)
    
    try:
        
        b = input('자본금을 입력하세요:')  # 자본금 입력
        b = int(b)        
                
        recommend_df['b'] = b
        
        recommend_df['b_ratio_high'] = b / formula1_p                         # 자본금 대비 매물가격의 비율 컬럼 생성 
        recommend_df['b_ratio_low'] = b / formula1_m
       
        recommend_df['ham_per_high'] = recommend_df['price']/formula1_p*100   # 예상 낙찰가 비율 컬럼 생성
        recommend_df['ham_per'] = recommend_df['price']/auc_pred*100
        recommend_df['ham_per_low'] = recommend_df['price']/formula1_m*100
        
        recommend_df['diff_low'] = trans_pred - formula1_m                    # 낙찰가 대비 실거래가 차익 컬럼 생성
        recommend_df['diff_high'] = trans_pred - formula1_p
        
        
        # 여러 조건에 따른 경쟁도 측정 컬럼 생성
        for step in range(len(recommend_df)):
            if recommend_df['ham_per_high'][step] >= 100:
                recommend_df.loc[step,'commpetition'] = '치열'
                
            elif recommend_df['ham_per'][step] >= 100:
                recommend_df.loc[step,'commpetition'] = '경계'
                
            else:
                recommend_df.loc[step,'commpetition'] = 'cool'
        
        
        # 여러 조건에 따른 예상 예산안 컬럼 생성
        for step in range(len(recommend_df)):
            if recommend_df['b_ratio_high'][step] >= 100 and recommend_df['b_ratio_low'][step] >= 100:
                recommend_df.loc[step,'pred_budget'] = '적정'
                
            elif recommend_df['b_ratio_high'][step] >= 100 and recommend_df['b_ratio_low'][step] <= 100:
                recommend_df.loc[step,'pred_budget'] = '도전'
                    
            else:
                recommend_df.loc[step,'pred_budget'] = '도박'
                
        print(recommend_df) 
        
        
        # 여러 조건에 따른 추천 투자방향 컬럼 생성
        for step in range(len(recommend_df)):
            if recommend_df['pred_budget'][step] in '적정' and recommend_df['commpetition'][step] in 'cool':
                recommend_df.loc[step,'recommend'] = '1'
            
            elif recommend_df['pred_budget'][step] in '적정' and recommend_df['commpetition'[step]] in '경계':
                recommend_df.loc[step,'recommend'] = '2'
            
            elif recommend_df['pred_budget'][step] in '적정' and recommend_df['commpetition'][step] in '치열':
                recommend_df.loc[step,'recommend'] = '3'
                
            elif recommend_df['pred_budget'][step] in '도전' and recommend_df['commpetition'][step] in 'cool':
                recommend_df.loc[step,'recommend'] = '2'
            
            elif recommend_df['pred_budget'][step] in '도전' and recommend_df['commpetition'][step] in '경계':
                recommend_df.loc[step,'recommend'] = '3'
            
            elif recommend_df['pred_budget'][step] in '도전' and recommend_df['commpetition'][step] in '치열':
                recommend_df.loc[step,'recommend'] = '4'
            
            elif recommend_df['pred_budget'][step] in '도박' and recommend_df['commpetition'][step] in 'cool':
                recommend_df.loc[step,'recommend'] = '3'
                    
            elif recommend_df['pred_budget'][step] in '도박' and recommend_df['commpetition'][step] in '경계':
                recommend_df.loc[step,'recommend'] = '4'
            
            elif recommend_df['pred_budget'][step] in '도박' and recommend_df['commpetition'][step] in '치열':
                recommend_df.loc[step,'recommend'] = '5'
                
        print(recommend_df)    
                
#         for step in range(len(recommend_df)):
#             if int(recommend_df['diff_high'][step]) >= 500000000 and recommend_df['pred_budget'][step] in '도박':
#                 recommend_df.loc[step,'soul'] = '영끌추천'
                
#             else:
#                 recommend_df.loc[step,'soul'] = '영끌비추천'
                    
                
            
    except TypeError:
        pass
      
           
    while True:
        print('번호나 이름을 입력하시오. \n1.완벽주의 투자자 \n2.신중하 투자자 \n3.적극적인 투자자 \n4.모험심 많은 투자자 \n5.공격적인 투자자')
        Investment_type = input('투자타입을 설정하세요:')
        
        if Investment_type in '1':
            type_items(Investment_type)
            
            print('확인을 원하시면 1번을 누르세요. \n뒤로 가시려면 아무키나 누르세요. \n종료를 원하시면 2번을 누르세요.')
            soul = input('적정가격 이외의 고수익 매물을 확인하시겠습니까?')
            
            # soul_pull함수 호출
            soul_pull(soul)
                        
        elif Investment_type in '2':
            type_items(Investment_type)
            
            print('확인을 원하시면 1번을 누르세요. \n뒤로 가시려면 아무키나 누르세요. \n종료를 원하시면 2번을 누르세요.')    
            soul = input('적정가격 이외의 고수익 매물을 확인하시겠습니까?')
            
            # soul_pull함수 호출
            soul_pull(soul)
                    
        elif Investment_type in '3':
            type_items(Investment_type)
            
            print('확인을 원하시면 1번을 누르세요. \n뒤로 가시려면 아무키나 누르세요. \n종료를 원하시면 2번을 누르세요.')        
            soul = input('적정가격 이외의 고수익 매물을 확인하시겠습니까?')
             
            # soul_pull함수 호출
            soul_pull(soul)
        
        elif Investment_type in '4':
            type_items(Investment_type)
            
            print('확인을 원하시면 1번을 누르세요. \n뒤로 가시려면 아무키나 누르세요. \n종료를 원하시면 2번을 누르세요.')        
            soul = input('적정가격 이외의 고수익 매물을 확인하시겠습니까?')
            
            # soul_pull함수 호출
            soul_pull(soul)
            
        elif Investment_type in '5':
            type_items(Investment_type)
            
            print('확인을 원하시면 1번을 누르세요. \n뒤로 가시려면 아무키나 누르세요. \n종료를 원하시면 2번을 누르세요.')        
            soul = input('적정가격 이외의 고수익 매물을 확인하시겠습니까?')
            
            # soul_pull함수 호출
            soul_pull(soul)
      
        else:
            print('잘못 입력되었습니다.')
                
                
 
        

In [142]:
df = pd.read_csv('../data/auction/auction_pjs3.csv')
recommend_apt(df)

INFO:tensorflow:Restoring parameters from model/auction/lys_auction_model_1_0.ckpt
Tensor("add_47:0", shape=(?, 1), dtype=float32)
예상 매매날짜(연,월)를 YYYYMM 형태로 기입하세요.202206
자본금을 입력하세요:1000000000
      office         num  case_year  cat  \
0     서울동부1계  2020-55340       2020  아파트   
1     서울동부4계  2020-55715       2020  아파트   
2     서울서부6계  2019-52798       2019  아파트   
3     서울서부3계  2019-54121       2019  아파트   
4     서울서부6계   2020-2843       2020  아파트   
...      ...         ...        ...  ...   
1090  서울중앙4계     2018-92       2018  아파트   
1091  서울남부4계    2018-609       2018  아파트   
1092  서울북부4계    2018-619       2018  아파트   
1093  서울북부6계    2018-633       2018  아파트   
1094  서울북부6계   2018-1117       2018  아파트   

                                                 addr  \
0     서울특별시 성동구 성수동1가 718, 103동 29층2902호 (성수동1가,트리마제)   
1       서울특별시 성동구 홍익동 53, 101동 10층1005호 (홍익동,청계대주파크빌)   
2       서울특별시 서대문구 연희동 194-30, 3층비312호 (연희동,브라운스톤 연희)   
3       서울특별시 서대문구 연희동 194-30, 3층비314호 (연희동,브라운스톤 연희

KeyError: 'c'

In [25]:
b = input('자본금을 입력하세요:')  # 자본금 입력
if b in ['\d+'] :
    print('a')

자본금을 입력하세요:20000


In [ ]:
'([가-힣]+구)'

In [68]:
df2 = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
                   columns=['a', 'b', 'c'])
df2
df2[['a','b']]


,a,b
0,1,2
1,4,5
2,7,8


In [105]:
a = df2['a']
print(type(a))
#for i in range(len(df2)):
#    for j,k in [float(df2['a'][i]), float(df2['b'][i])]:
#        print(j,k)
a = a[1]
print(type(a))
print(type(float(df2['a'][1])))


<class 'pandas.core.series.Series'>
<class 'numpy.int32'>
<class 'float'>


In [118]:
for i in range(len(df2)):
    for j,k in zip[a, b]:
        print(j,k)

TypeError: 'type' object is not subscriptable

<function ndarray.flatten>

In [104]:
float(df2['a'][1])

4.0

In [ ]:
3456 /5

In [ ]:
df2.loc[1,'d']='적정'
df2
print('gkdl',df2['a'][0],',',df2['b'][0],df2['c'][0])

In [116]:
a = df2['a']
b = df2['b']
print('aa:',list(df2.loc[1,['a','b']]))
a = a.values 
b = b.values

aa: [4, 5]


In [ ]:
#while True:
def a(x):
    #a = input('dd:')
    a = x
    if a == 1:
        for i in range(len(df2)):
            print(df2.loc[i,'a'])

In [ ]:
print(a(2))